In [49]:
from surprise import KNNBaseline, Reader
from surprise import Dataset
import json
import pickle
from nltk import FreqDist

# 全局变量
file = 'data.txt'
line_format = 'user item rating'
sep = '\t'

def read(file):
    f = open(file,'r',encoding='utf8')
    data = [json.loads(i) for i in f.readlines()]
    f.close()
    return data

In [2]:
id_to_name = pickle.load(open('id_to_name.pkl','rb'))
print('加载歌曲id到歌曲名的映射完成...')
name_to_id = {id_to_name[iid]: iid for iid in id_to_name}
print('加载歌曲名到歌曲id的映射完成...')

加载歌单id到歌单名的映射完成...
加载歌单名到歌单id的映射完成...


In [3]:
reader = Reader(line_format=line_format, sep=sep)
data = Dataset.load_from_file(file, reader=reader)
trainset = data.build_full_trainset()
print('数据集构建完成...')

数据集构建完成...


In [5]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.train(trainset)

D:\softwares\Anaconda\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [10]:
pickle.dump(algo, open('model.pkl','wb'))

In [13]:
def recommond(sid):
    print('为歌曲《%s》推荐：'%id_to_name[sid])
    inner_id = trainset.to_inner_iid(str(sid))

    # 找到最近的10个邻居
    neighbors = (algo.get_neighbors(inner_id, k=10))
    # 打印对应的item名称
    neighbors = [id_to_name[int(trainset.to_raw_iid(n))] for n in neighbors]
    
    print(neighbors)
    print()

In [16]:
recommond(471403427)
recommond(181946)
recommond(name_to_id['空空如也'])
recommond(562594267)

为歌曲《我喜欢上你时的内心活动》推荐：
['岁月神偷', '春风十里', '追光者', '我要你', '致姗姗来迟的你 ', '小半', '可乐', '我多喜欢你，你会知道', '就是爱你', '红色高跟鞋']

为歌曲《纤夫的爱》推荐：
['青青河边草', '千年等一回', '知心爱人', '上海滩', '中华民谣', '懂你', '窗外', '众人划桨开大船', '天不下雨天不刮风天上有太阳', '心雨']

为歌曲《空空如也》推荐：
['非酋', '岁月神偷', '说散就散', '小半', '追光者', '我唯一青春里的路人', '不找了', '其实，我就在你方圆几里（Cover 薛之谦）', '暧昧', '再也没有']

为歌曲《成全》推荐：
['说谎', '我们', '可乐', '你就不要想起我', '戒烟', '哑巴', '你，好不好？', '浪费', '后来的我们', '空空如也 ']



In [45]:
pid_to_sid = pickle.load(open('playlist.pkl','rb'))
print('加载歌单id到包含的歌曲id列表的映射完成...')
sid_to_pid = dict()
for pid in pid_to_sid:
    for s in pid_to_sid[pid]:
        if s not in sid_to_pid:
            sid_to_pid[s] = []
        sid_to_pid[s].append(pid)
print('加载歌曲id到歌单id列表的映射完成...')

加载歌单id到包含的歌曲id列表的映射完成...
加载歌曲id到歌单id列表的映射完成...


In [4]:
sim_options = {'name': 'pearson_baseline', 'user_based': True}
algo = KNNBaseline(sim_options=sim_options)
algo.train(trainset)
pickle.dump(algo, open('userCF.pkl','wb'))

D:\softwares\Anaconda\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [61]:
def recommond_playlist(pid):
    print('为id为《%s》的歌单推荐的其他歌单的id：'%str(pid))
    uid = trainset.to_inner_uid(str(pid))
    neighbors = algo.get_neighbors(uid,k=10)
    print([trainset.to_raw_uid(n) for n in neighbors])
    print()

def recommond_songs(sid):
    print('为歌曲《%s》推荐：'%id_to_name[sid])
    pids = sid_to_pid[sid]
    neighbors = []
    for pid in pids:
        uid = trainset.to_inner_uid(str(pid))
        for neigh in algo.get_neighbors(uid,k=10):
            neighbors += pid_to_sid[int(trainset.to_raw_uid(neigh))]
    neighbors = [id_to_name[i] for i,_ in FreqDist(neighbors).most_common()[:10]]
    print(neighbors)
    print()

In [55]:
recommond_playlist(2239964825)

为id为《2239964825》的歌单推荐的其他歌单的id：
['1991114941', '723552791', '2147373807', '2124336941', '778008332', '942833866', '982358515', '906995729', '611796201', '2170719494']



In [62]:
recommond_songs(471403427)
recommond_songs(181946)
recommond_songs(name_to_id['空空如也'])
recommond_songs(562594267)

为歌曲《我喜欢上你时的内心活动》推荐：
['小半', '岁月神偷', '追光者', '就是爱你', '遇见', '春风十里', '理想三旬', '可乐', '红色高跟鞋', '我喜欢上你时的内心活动']

为歌曲《纤夫的爱》推荐：
['风中有朵雨做的云', '心雨', '新鸳鸯蝴蝶梦', '懂你', '涛声依旧', '偏偏喜欢你', '潇洒走一回', '青青河边草', '我想有个家', '月亮代表我的心']

为歌曲《空空如也》推荐：
['小半', '岁月神偷', '可乐', '追光者', '过客', '白羊', '暧昧', '起风了（Cover 高橋優）', '演员', '你，好不好？']

为歌曲《成全》推荐：
['岁月神偷', '可乐', '小半', '你，好不好？', '你还要我怎样', '追光者', '暧昧', '过客', '白羊', '说谎']

